In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [1]:
import torch
torch.cuda.empty_cache()

# **Install dependencies**

In [7]:
%%capture
!! pip install keras
!! pip install netron
!! pip install nltk

!! pip install git+https://github.com/huggingface/transformers
!! pip install datasets
!! pip install nlp
!! pip install rouge-score

!! pip install git-python
!! pip install sentencepiece

!! pip install sacrebleu==1.5.1
!! pip install tree-sitter
!! pip install evaluate

In [ ]:
!! git clone https://github.com/huggingface/evaluate.git
%cd evaluate
!! pip install -e .

In [ ]:
!! pip install torchdata
!! pip install transformers
!! pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 24.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# **Import Libraries**

In [5]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


ModuleNotFoundError: No module named 'datasets'

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
%cd /content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian

[Errno 2] No such file or directory: '/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian'
/content


# **Load Data**

In [6]:
import datasets
import pandas as pd

from datasets import Dataset
from datasets import load_dataset


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import json

In [90]:
# Load the Excel file
df = pd.read_excel("Internership_TestCase.xlsx")

# Split data into train (80%), validation (10%), and test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert pandas DataFrames to lists of dictionaries
train_data = train_df.to_dict(orient='records')
val_data = val_df.to_dict(orient='records')
test_data = test_df.to_dict(orient='records')

# Create a dictionary with the splits
dataset_dict = {
    'train': train_data,
    'validation': val_data,
    'test': test_data
}

# Save the dictionary to a JSON file
save_path = "./dataset.json"
with open(save_path, 'w') as f:
    json.dump(dataset_dict, f)

print(f"Dataset saved successfully at {save_path}!")




Dataset saved successfully at ./dataset.json!
Dataset loaded successfully from ./dataset.json!


In [8]:
# Load the dictionary from the JSON file
with open("./dataset.json", 'r') as f:
    dataset_dict = json.load(f)

# Convert the lists of dictionaries back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(pd.DataFrame(dataset_dict['train']))
val_dataset = Dataset.from_pandas(pd.DataFrame(dataset_dict['validation']))
test_dataset = Dataset.from_pandas(pd.DataFrame(dataset_dict['test']))

# Create a DatasetDict
loaded_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})




In [9]:
# Access the datasets
train_dataset = loaded_dataset['train']
val_dataset = loaded_dataset['validation']
test_dataset = loaded_dataset['test']

In [10]:
train_dataset[1]

{'Actions': 'Click on the Favorite List icon',
 'Code Groovy': "WebUI.click(findTestObject('Wave2/XPath', [('path') : '//span[@class=\\'x-btn-icon-el x-btn-icon-el-default-small icon icon-favorite favorite-top \\']']))"}

In [11]:
loaded_dataset

DatasetDict({
    train: Dataset({
        features: ['Actions', 'Code Groovy'],
        num_rows: 166
    })
    validation: Dataset({
        features: ['Actions', 'Code Groovy'],
        num_rows: 21
    })
    test: Dataset({
        features: ['Actions', 'Code Groovy'],
        num_rows: 21
    })
})

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("AhmedSSoliman/DJANGO")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1805 [00:00<?, ? examples/s]

In [ ]:
#train_dataset= raw_dataset["train"]
#val_dataset = raw_dataset["validation"]
#test_dataset = raw_dataset["test"]

In [ ]:
#raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1805
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'nl', 'code'],
        num_rows: 1000
    })
})

# **Tokenizer**

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base", use_fast = True)
tokenizer
# "Helsinki-NLP/opus-mt-en-nl"
# "distilroberta-base"
##### "studio-ousia/luke-base"
#"AhmedSSoliman/MarianCG_NL-to-Code"
#"distilbert-base-uncased"
#studio-ousia/mluke-base-lite
# "google/electra-small-discriminator"
#"bert-base-uncased"
#"microsoft/deberta-v3-xsmall"
#"microsoft/deberta-v3-base"
# "xlnet-base-cased"
#"google/fnet-base"
#"google/bigbird-roberta-base"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RobertaTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

# **Model**

In [ ]:
#All models
#model = EncoderDecoderModel.from_encoder_decoder_pretrained("distilroberta-base", "Helsinki-NLP/opus-mt-en-nl")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("distilbert-base-uncased", "Helsinki-NLP/opus-mt-en-nl")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("google/electra-base-discriminator", "Helsinki-NLP/opus-mt-en-nl")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("studio-ousia/luke-base", "Helsinki-NLP/opus-mt-en-nl")
# model = EncoderDecoderModel.from_encoder_decoder_pretrained("distilroberta-base", "distilgpt2")
#"distilgpt2"
#"studio-ousia/luke-base"
#"distilroberta-base"
#"google/electra-base-discriminator"
#"bert-base-uncased"
#"xlnet-base-cased"
#"microsoft/deberta-v3-xsmall"
#"christina/decoder-only-transformer-x6small"

In [13]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained("distilroberta-base", "Helsinki-NLP/opus-mt-en-nl")
model

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [14]:
model.config

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.1,
    "early_stopping": false,
    "encoder_attention_heads": 8,
    "encoder_ffn_dim": 2048,
    "encoder_lay

In [15]:
model.base_model.config.encoder.num_hidden_layers = 6

In [16]:
model.tie_weights = True

In [17]:
model.base_model.config.decoder_start_token_id = model.base_model.config.decoder.bos_token_id

In [18]:
model.base_model.config.bos_token_id = tokenizer.bos_token_id
model.base_model.config.eos_token_id = tokenizer.eos_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

In [19]:
model.config.max_length =512

In [20]:
model.save_pretrained("distilroberta_marian_base_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [ ]:
model=AutoModelForSeq2SeqLM.from_pretrained("./distilroberta_marian_training")


In [24]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./distilroberta_marian_base_model")

model

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [25]:
model.base_model.config.decoder_start_token_id

0

In [26]:
model.base_model.config.pad_token_id

1

In [27]:
tokenizer.pad_token_id

1

In [28]:
model.config

EncoderDecoderConfig {
  "_name_or_path": "./distilroberta_marian_base_model",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "bos_token_id": 0,
  "decoder": {
    "_name_or_path": "Helsinki-NLP/opus-mt-en-nl",
    "_num_labels": 3,
    "activation_dropout": 0.0,
    "activation_function": "swish",
    "add_bias_logits": false,
    "add_cross_attention": true,
    "add_final_layer_norm": false,
    "architectures": [
      "MarianMTModel"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": [
      [
        67027
      ]
    ],
    "begin_suppress_tokens": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classif_dropout": 0.0,
    "classifier_dropout": 0.0,
    "cross_attention_hidden_size": null,
    "d_model": 512,
    "decoder_attention_heads": 8,
    "decoder_ffn_dim": 2048,
    "decoder_layerdrop": 0.0,
    "decoder_layers": 6,
    "decoder_start_token_id": 67027,
    "decoder_vocab_size": 67028,
    "diversity_penalty": 0.0,
    "do_sample": false,

# **Evaluator**

In [36]:
%cd /content/distilroberta_marian_base_model/Utils

/content/distilroberta_marian_base_model/Utils


In [49]:
!git clone https://github.com/huggingface/evaluate.git
%cd evaluate
!pip install -e .

Cloning into 'evaluator'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'evaluator'
/content/distilroberta_marian_base_model/Utils/evaluate
Obtaining file:///content/distilroberta_marian_base_model/Utils/evaluate
  Preparing metadata (setup.py) ... done
  Attempting uninstall: evaluate
    Found existing installation: evaluate 0.4.2
    Uninstalling evaluate-0.4.2:
      Successfully uninstalled evaluate-0.4.2
  Running setup.py develop for evaluate


In [48]:
import evaluate

In [56]:
from metrics import load_metric


ImportError: cannot import name 'load_metric' from 'metrics' (unknown location)

In [59]:
from evaluate import metrics


In [62]:
from metrics import sacrebleu

In [55]:
Metric = load_metric('sacrebleu')

NameError: name 'load_metric' is not defined

In [63]:
from google.colab import files
evaluator = metrics.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

AttributeError: module 'evaluate.metrics' has no attribute 'CodeGenerationEvaluator'

In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian




# **Prepare the data.**

In [29]:
encoder_length = 32
decoder_length = 32
batch_size = 1


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["Actions"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["Code Groovy"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [30]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['Actions', 'Code Groovy'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



Map:   0%|          | 0/166 [00:00<?, ? examples/s]

In [1]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['Actions', 'Code Groovy'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


NameError: name 'val_dataset' is not defined

In [40]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

# **Training**

In [49]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./distilroberta-marian-training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=10,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1504: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [48]:
!git clone https://github.com/huggingface/accelerate
%cd accelerate
!pip install -e .

Cloning into 'accelerate'...
remote: Enumerating objects: 13253, done.
remote: Counting objects: 100% (4833/4833), done.
remote: Compressing objects: 100% (553/553), done.
remote: Total 13253 (delta 4511), reused 4360 (delta 4246), pack-reused 8420
Receiving objects: 100% (13253/13253), 4.37 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (9191/9191), done.
/content/distilroberta_marian_base_model/Utils/accelerate
Obtaining file:///content/distilroberta_marian_base_model/Utils/accelerate
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for accelerate (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.32.0.dev0-0.editable-py3-none-any.whl size=12178 sha256=11d33eafd43071e2941dbafbfbc600462c8ba55c584f9a6914e5ad95ab7ce190
  Stored in directory: /tmp/pip-ephem-wheel-cache-rfyl_rd3/wheel

In [ ]:
# distilRoBERTa-Marian-Training
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 160000
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,1.259300,1.112706,12.093183,11.579566,31.111363,16.870151,8.920128,15.495665,46.322058,31.125087,15.391544,8.305034
2,0.703100,0.677957,41.742606,44.027506,65.590112,48.593888,35.766605,41.896389,68.156894,66.294416,49.262837,38.177507
3,0.550000,0.509812,54.167319,56.049901,72.915196,59.627677,49.061595,57.159857,78.445924,73.176988,60.185559,51.079781
4,0.404900,0.427548,64.497157,66.055326,80.985618,69.347209,59.596095,64.131359,82.926478,80.996735,69.736563,61.410788
5,0.354300,0.375074,67.809657,69.669556,85.101928,74.699690,66.121759,67.807849,85.393115,85.603544,75.808799,68.707569
6,0.288900,0.353043,70.715418,72.363218,85.982832,76.520785,68.230897,70.014224,86.724613,86.516071,77.765499,70.842627
7,0.246200,0.336535,72.808929,74.193876,87.822066,79.225439,71.688019,71.528419,87.857990,88.340898,80.291643,74.117944
8,0.237300,0.322036,73.595702,75.128414,87.367050,78.658388,71.495715,72.065756,88.114859,87.716691,79.659153,73.703019
9,0.211600,0.313317,74.852383,76.121581,87.902090,79.727924,72.759857,72.908647,88.657585,88.274358,80.712950,74.919852
10,0.205200,0.313937,75.167198,76.455672,88.299956,80.385931,73.533084,73.393079,88.900018,88.875598,81.454827,75.773004


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1
Saving model checkpoint to ./distilroberta-marian-training/checkpoint-16000
Configuration saved in ./distilroberta-marian-training/checkpoint-16000/config.json
Model weights saved in ./distilroberta-marian-training/checkpoint-16000/pytorch_model.bin
tokenizer config file saved in ./distilroberta-marian-training/checkpoint-16000/tokenizer_config.json
Special tokens file saved in ./distilroberta-marian-training/checkpoint-16000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2317: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_lengt

TrainOutput(global_step=160000, training_loss=0.617371564245224, metrics={'train_runtime': 13782.5148, 'train_samples_per_second': 11.609, 'train_steps_per_second': 11.609, 'total_flos': 2111609241600000.0, 'train_loss': 0.617371564245224, 'epoch': 10.0})

In [ ]:
# distilRoBERTa-Marian-Training-old
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 160000
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.267400,0.327421,73.834146,75.400138,87.714721,79.129056,71.865909,72.504967,88.370630,87.781732,79.870557,73.810987
2,0.193600,0.310745,75.361611,76.756218,88.220579,80.485721,73.762994,73.958711,88.990059,88.419592,81.322914,75.749352
3,0.170000,0.302277,76.961354,78.115608,88.083564,80.326891,73.898946,74.702454,89.584204,88.194647,81.121294,75.684147
4,0.137100,0.290385,76.975630,78.478887,87.635092,80.007111,73.545186,75.174441,89.777397,87.882887,80.885656,75.516788
5,0.131400,0.285707,77.774790,79.146762,89.587191,82.581591,76.656020,76.107073,90.268323,89.641513,83.304023,78.393693
6,0.100000,0.285543,79.110200,80.092983,91.315383,85.003721,79.313753,76.604302,90.700086,91.418211,85.595878,81.089582
7,0.085500,0.278289,79.182430,80.511640,90.082555,83.658299,78.106836,76.781054,90.757730,90.432282,84.624697,80.150506


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1
Saving model checkpoint to ./distilroberta-marian-training1/checkpoint-16000
Configuration saved in ./distilroberta-marian-training1/checkpoint-16000/config.json
Model weights saved in ./distilroberta-marian-training1/checkpoint-16000/pytorch_model.bin
tokenizer config file saved in ./distilroberta-marian-training1/checkpoint-16000/tokenizer_config.json
Special tokens file saved in ./distilroberta-marian-training1/checkpoint-16000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2317: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.267400,0.327421,73.834146,75.400138,87.714721,79.129056,71.865909,72.504967,88.370630,87.781732,79.870557,73.810987
2,0.193600,0.310745,75.361611,76.756218,88.220579,80.485721,73.762994,73.958711,88.990059,88.419592,81.322914,75.749352
3,0.170000,0.302277,76.961354,78.115608,88.083564,80.326891,73.898946,74.702454,89.584204,88.194647,81.121294,75.684147
4,0.137100,0.290385,76.975630,78.478887,87.635092,80.007111,73.545186,75.174441,89.777397,87.882887,80.885656,75.516788
5,0.131400,0.285707,77.774790,79.146762,89.587191,82.581591,76.656020,76.107073,90.268323,89.641513,83.304023,78.393693
6,0.100000,0.285543,79.110200,80.092983,91.315383,85.003721,79.313753,76.604302,90.700086,91.418211,85.595878,81.089582
7,0.085500,0.278289,79.182430,80.511640,90.082555,83.658299,78.106836,76.781054,90.757730,90.432282,84.624697,80.150506
8,0.094100,0.273894,79.824580,80.932717,90.060567,83.858841,78.543903,77.037467,90.799239,89.974420,84.340899,80.058651
9,0.082500,0.272633,79.768941,80.984759,89.271580,82.896155,77.527480,76.989931,90.819831,89.348652,83.550858,79.182246
10,0.077600,0.274617,80.256615,81.401178,90.100291,83.935694,78.801591,77.334802,91.046787,90.233107,84.695102,80.591181


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1
Saving model checkpoint to ./distilroberta-marian-training1/checkpoint-128000
Configuration saved in ./distilroberta-marian-training1/checkpoint-128000/config.json
Model weights saved in ./distilroberta-marian-training1/checkpoint-128000/pytorch_model.bin
tokenizer config file saved in ./distilroberta-marian-training1/checkpoint-128000/tokenizer_config.json
Special tokens file saved in ./distilroberta-marian-training1/checkpoint-128000/special_tokens_map.json
Deleting older checkpoint [distilroberta-marian-training1/checkpoint-96000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2317: UserWarning: `max_length` 

TrainOutput(global_step=160000, training_loss=0.13924457960128783, metrics={'train_runtime': 13506.2835, 'train_samples_per_second': 11.846, 'train_steps_per_second': 11.846, 'total_flos': 2111609241600000.0, 'train_loss': 0.13924457960128783, 'epoch': 10.0})

In [ ]:
# distilRoBERTa-Marian-Training-1
trainer.train()

The following columns in the training set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16000
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 160000
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `

Epoch,Training Loss,Validation Loss,Bleu,Sacrebleu,Bleu-unigram-precision,Bleu-bigram-precision,Bleu-trigram-precision,Rouge-2,Rouge-l,Sacre-unigram-precision,Sacre-bigram-precision,Sacre-trigram-precision
1,0.267400,0.327421,73.834146,75.400138,87.714721,79.129056,71.865909,72.643151,88.442203,87.781732,79.870557,73.810987
2,0.193600,0.310745,75.361611,76.756218,88.220579,80.485721,73.762994,74.095062,89.050357,88.419592,81.322914,75.749352
3,0.170000,0.302277,76.961354,78.115608,88.083564,80.326891,73.898946,74.809115,89.608955,88.194647,81.121294,75.684147
4,0.137100,0.290385,76.975630,78.478887,87.635092,80.007111,73.545186,75.317715,89.815351,87.882887,80.885656,75.516788
5,0.131400,0.285707,77.774790,79.146762,89.587191,82.581591,76.656020,76.216947,90.303216,89.641513,83.304023,78.393693
6,0.100000,0.285543,79.110200,80.092983,91.315383,85.003721,79.313753,76.722190,90.742864,91.418211,85.595878,81.089582
7,0.085500,0.278289,79.182430,80.511640,90.082555,83.658299,78.106836,76.926052,90.804006,90.432282,84.624697,80.150506
8,0.094100,0.273894,79.824580,80.932717,90.060567,83.858841,78.543903,77.160399,90.859090,89.974420,84.340899,80.058651
9,0.082500,0.272633,79.768941,80.984759,89.271580,82.896155,77.527480,77.136882,90.873266,89.348652,83.550858,79.182246
10,0.077600,0.274617,80.256615,81.401178,90.100291,83.935694,78.801591,77.497709,91.114060,90.233107,84.695102,80.591181


The following columns in the evaluation set don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: Unnamed: 0. If Unnamed: 0 are not expected by `EncoderDecoderModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1
Saving model checkpoint to ./distilroberta-marian-training1/checkpoint-16000
Configuration saved in ./distilroberta-marian-training1/checkpoint-16000/config.json
Model weights saved in ./distilroberta-marian-training1/checkpoint-16000/pytorch_model.bin
tokenizer config file saved in ./distilroberta-marian-training1/checkpoint-16000/tokenizer_config.json
Special tokens file saved in ./distilroberta-marian-training1/checkpoint-16000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2319: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_

TrainOutput(global_step=160000, training_loss=0.13924457960128783, metrics={'train_runtime': 13035.4538, 'train_samples_per_second': 12.274, 'train_steps_per_second': 12.274, 'total_flos': 2111609241600000.0, 'train_loss': 0.13924457960128783, 'epoch': 10.0})

In [ ]:
trainer.save_model()
trainer.save_state()

Saving model checkpoint to ./distilroberta-marian-training1
Configuration saved in ./distilroberta-marian-training1/config.json
Model weights saved in ./distilroberta-marian-training1/pytorch_model.bin
tokenizer config file saved in ./distilroberta-marian-training1/tokenizer_config.json
Special tokens file saved in ./distilroberta-marian-training1/special_tokens_map.json


# **Visualize**

In [ ]:
%load_ext tensorboard

### **distilRoBERTa-marian**

In [ ]:
tensorboard --logdir ./distilroberta-Marian-Training/runs

<IPython.core.display.Javascript object>

# **Inference**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained("./distilroberta-marian-training1/")
new_model.to("cuda")

new_tokenizer = AutoTokenizer.from_pretrained("./distilroberta-marian-training1/", use_fast=True)


In [ ]:
new_model

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [ ]:
new_model.config.max_length


512

# **Evaluator**

In [ ]:
%cd Utils

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian/Utils


In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(new_tokenizer, device,smooth_bleu=True)

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian/Utils/evaluator.py:32: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.sacre_bleu: Metric = load_metric('sacrebleu')


In [ ]:
batch_size = 1

# map data correctly
def generate_new_code(batch):
    inputs = new_tokenizer(batch["nl"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_code"] = output_str

    return batch


In [ ]:
results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size)

#results = test_dataset.map(generate_new_code, batched=True, batch_size=batch_size, remove_columns=["intent"])

  0%|          | 0/1805 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1261: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian


In [ ]:
import re
import string

import datasets
import numpy as np

import evaluate
from evaluate import load

In [ ]:

exact_match = evaluate.load("exact_match")

#refs = ["The cat sat on the mat.", "Theaters are great.", "It's like comparing oranges and apples."]
#preds = ["The cat sat on the mat?", "Theaters are great.", "It's like comparing apples and oranges."]

"""
for i in range(len(results)):
  refs = results["code"][i]
  preds = results["pred_code"][i]
"""

'\nfor i in range(len(results)):\n  refs = results["code"][i]\n  preds = results["pred_code"][i]\n'

In [ ]:
refs = results["code"]
preds = results["pred_code"]

In [ ]:

exact_match_results = exact_match.compute(references=refs, predictions=preds, regexes_to_ignore=["▁", " "], ignore_case=True, ignore_punctuation=True, ignore_numbers=True)
print(round(exact_match_results["exact_match"], 5)*100)

77.95


In [ ]:
with open("./references.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(ref + '\n')
    else:
      continue


with open("./predictions.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":
        f.write(pred + '\n')
    else:
      continue


In [ ]:
# Ok Evaluator
bleu_score =0
rouge_score = 0
sacre_bleu = 0

for i in range(len(results)):
  ref = results["code"][i]
  pred = results["pred_code"][i]
  if pred is not None and pred != "":
      if ref is not None and ref != "":
        #bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
        #bleu_score  += bleu_metric["score"]

        bleu_metric = evaluator.evaluate([pred], [ref])
        #bleu_metric = evaluator.evaluateSingle(pred, ref)

        #bleu_score += bleu_metric['BLEU-Unigram-Precision']
        bleu_score += bleu_metric['BLEU']
        rouge_score += bleu_metric['ROUGE-L']
        sacre_bleu += bleu_metric['SacreBLEU']
        #print(bleu_metric['SacreBLEU'])
  else:
    continue

# **Performance Metrics**

**BLEU Score**

In [ ]:
# Marian  85.13
# print("Bleu Score: ",bleu_score/len(results))
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 88.91230715490435 (scale 0-100)


**SacreBLEU**

In [ ]:
# Marian  70.26
# print("Bleu Score: ",bleu_score/len(results))
print('Sacre Bleu: {} (scale 0-100)'.format(sacre_bleu/len(results)))

Sacre Bleu: 74.08303353280789 (scale 0-100)


**Rouge Score**

In [ ]:
#Marian  90.07
# print("Bleu Score: ",bleu_score/len(results))
print('ROUGE Score: {} (scale 0-100)'.format(rouge_score/len(results)))

ROUGE Score: 92.7350696084888 (scale 0-100)


**CodeBLEU**

In [ ]:
from google.colab import files

In [ ]:
import calc_code_bleu
ngram_match_score, weighted_ngram_match_score, syntax_match_score, dataflow_match_score, code_bleu_score = calc_code_bleu.calc_code_bleu("references.txt","predictions.txt","python", "keywords")

In [ ]:
print("ngram_match_score: ", ngram_match_score*100)
print("weighted_ngram_match_score: ", weighted_ngram_match_score*100)
print("syntax_match_score: ", syntax_match_score*100)
print("dataflow_match_score: ", dataflow_match_score*100)
print("code_bleu_score: ", code_bleu_score*100)

# **Save the  results**

In [ ]:
%cd ..

/content/drive/MyDrive/models/CodeGeneration/MarianCG-CoNaLa-Large


In [ ]:
pwd

'/content/drive/MyDrive/models/CodeGeneration/New/DJANGO/distilRoBERTa-Marian'

In [ ]:
bleu_score = 0
with open("./distilroberta-marian-training/Output.txt", 'w') as f:
  for i in range(len(results)):
    nl = results["nl"][i]
    ref = results["code"][i]
    pred = results["pred_code"][i]

    if pred is not None and pred != "":
      if ref is not None and ref != "":

        bleu_metric = evaluator.evaluate([pred], [ref])
        f.write( "NL: " + nl + '\n')
        f.write( "ref: " + ref + '\n')
        f.write( "Output: " + pred + '\n')
        f.write("bleu_metric: " + str(bleu_metric["BLEU"]) + '\n')
        f.write(10*"-" + '\n')
        bleu_score  += bleu_metric["BLEU"]

    else:
      continue


# **Test**

In [ ]:
# map data correctly
def generate_code(NL):
    inputs = new_tokenizer([NL], padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_code = new_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_code

In [ ]:

#NL = "decode a hex string to utf-8."
NL = "remove first and last lines of string `s`"
output_code = generate_code(NL)

In [ ]:
output_code

In [ ]:
natural_language = "How I can get rid of None values in dictionary?"
gen_code = generate_code(natural_language)

In [ ]:
gen_code